In [ ]:
!nvidia-smi

In [ ]:
!nvidia-smi -L

In [ ]:
!python --version

In [ ]:
train_dir = "/104_fruits_original/train/"
val_dir = "/104_fruits_original/val/"
test_dir = "/104_fruits_original/test/"

In [ ]:
save_model = "/save_model/checkpoint"

In [ ]:
# setup data for train, val and test
import tensorflow as tf
IMAGE_SHAPE = (224, 224)

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="categorical",
                                                                 image_size=IMAGE_SHAPE)
val_data = tf.keras.preprocessing.image_dataset_from_directory(val_dir,
                                                                 label_mode="categorical",
                                                                 image_size=IMAGE_SHAPE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                 label_mode="categorical",
                                                                 image_size=IMAGE_SHAPE,
                                                                 shuffle=False)

In [ ]:
# Create checkpoint callback to save model
checkpoint_path = "./feature_exaction/checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         monitor="val_accuracy",
                                                         save_best_only=True,
                                                         save_weights_only=True)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# # Setup data augmentation
# data_augmentation = Sequential([
#   preprocessing.RandomFlip("horizontal"),
#   preprocessing.RandomRotation(0.2),
#   preprocessing.RandomHeight(0.2),
#   preprocessing.RandomWidth(0.2),
#   preprocessing.RandomZoom(0.2), 
# #   preprocessing.Rescaling(1./255)
# ], name="data_augmentation")

In [ ]:
# Setup base model and freeze its layers
base_model = tf.keras.applications.DenseNet201(include_top=False)
base_model.trainable = False

# Setup model architecture with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name="input_layer") 
x = tf.keras.applications.densenet.preprocess_input(inputs)
# x = data_augmentation(x) 
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x) 
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

# Compile
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
# Fit model
history_feature_extract = model.fit(train_data,
                    epochs=4, 
                    validation_data=val_data,
                    validation_steps=len(val_data),
                    callbacks=[checkpoint_callback])

In [ ]:
# Evaluate model with val set
results_feature_extract_model = model.evaluate(val_data)
results_feature_extract_model

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [ ]:
plot_loss_curves(history_feature_extract)

In [ ]:
# Clone the model we created
cloned_model = tf.keras.models.clone_model(model)
cloned_model.summary()

In [ ]:
# Load checkpointed weights into cloned_model
cloned_model.load_weights(save_model)
# Compile cloned_model 
cloned_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
model = cloned_model

In [ ]:
# Evaluate trained model with test set
loaded_loss, loaded_accuracy = model.evaluate(test_data)
loaded_loss, loaded_accuracy

In [ ]:
# Make predictions with model
pred_probs = model.predict(test_data, verbose=1)

In [ ]:
# Get the class predicitons of each label
pred_classes = pred_probs.argmax(axis=1)

In [ ]:
y_labels = []
for images, labels in test_data.unbatch(): # unbatch the test data and get images and labels
    y_labels.append(labels.numpy().argmax()) # append the index which has the largest value (labels are one-hot)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_labels, pred_classes))

In [ ]:
model.save("model_1.h5")

# Fine-tuning

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 5 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                  patience=5) 

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "./fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, 
                                                 patience=3,
                                                 verbose=1, 
                                                 min_lr=1e-7)


In [ ]:
# Check layers in our base model
for layer_number, layer in enumerate(base_model.layers):
    print(layer_number, layer.name)

In [ ]:
# Check which layers are trainable
for layer_number, layer in enumerate(model.layers):
    print(layer_number, layer.name, layer.trainable)

In [ ]:
# Check which layers are trainable
for layer_number, layer in enumerate(model.layers[5].layers):
    print(layer_number, layer.name, layer.trainable)

In [ ]:
model.summary()

In [ ]:
# Unfreeze all of the layers in the base model
base_model.trainable = True

# Refreeze layer
for layer in base_model.layers[:-240]:
    layer.trainable = False

In [ ]:
# Check which layers are trainable
for layer_number, layer in enumerate(model.layers[5].layers):
    print(layer_number, layer.name, layer.trainable)

In [ ]:
model.summary()

In [ ]:
# Recompile model 
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# Fine-tune
history_fine_tune = model.fit(train_data,
                              epochs=350,
                              steps_per_epoch=len(val_data),
                              validation_data=val_data,
                              validation_steps=len(val_data),
                              callbacks=[
                                  model_checkpoint, 
                                         early_stopping, 
                                         reduce_lr])

In [ ]:
# Evaluate fine-tuned model on the whole test dataset
results_all_fine_tune = model.evaluate(val_data)
results_all_fine_tune

In [ ]:
plot_loss_curves(history_fine_tune)

In [ ]:
# Evaluate trained model with test set
loaded_loss, loaded_accuracy = model.evaluate(test_data)
loaded_loss, loaded_accuracy

In [ ]:
# Make predictions with model
pred_probs = model.predict(test_data, verbose=1)

In [ ]:
# Get the class predicitons of each label
pred_classes = pred_probs.argmax(axis=1)

In [ ]:
y_labels = []
for images, labels in test_data.unbatch(): # unbatch the test data and get images and labels
    y_labels.append(labels.numpy().argmax()) # append the index which has the largest value (labels are one-hot)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_labels, pred_classes))

In [ ]:
model.save("model_2.h5")